In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [6]:
# Path to the replicated train data
data_path = r"D:\mini project\Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning\data\train_replicated_with_dates.csv"


# Load the data
train_data = pd.read_csv(data_path)
print(f"Data loaded with {len(train_data)} rows.")

# Split into features (X) and target (y)
X = train_data.drop(columns=["id", "num_orders", "date"])  # Exclude ID, date, and target
y = train_data["num_orders"]  # Target variable

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the train-test split for later use
X_train.to_csv(r"D:\mini project\Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning\data\X_train.csv", index=False)
X_val.to_csv(r"D:\mini project\Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning\data\X_val.csv", index=False)
y_train.to_csv(r"D:\mini project\Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning/data/y_train.csv", index=False)
y_val.to_csv(r"D:\mini project\Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning\data\y_val.csv", index=False)

print(f"Train and validation sets saved. Train: {len(X_train)}, Validation: {len(X_val)}.")


Data loaded with 2000000 rows.
Train and validation sets saved. Train: 1600000, Validation: 400000.


In [11]:
import pandas as pd
from xgboost import XGBRegressor
import os

# Set the base directory where the files are located
base_dir = r"D:/mini project/Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning/data"

# Verify if files exist and load them
try:
    # Construct file paths
    X_train_path = os.path.join(base_dir, "X_train.csv")
    y_train_path = os.path.join(base_dir, "y_train.csv")
    X_val_path = os.path.join(base_dir, "X_val.csv")
    y_val_path = os.path.join(base_dir, "y_val.csv")
    
    # Ensure files exist
    for file_path in [X_train_path, y_train_path, X_val_path, y_val_path]:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
    
    # Load datasets
    X_train = pd.read_csv(X_train_path)
    y_train = pd.read_csv(y_train_path).squeeze()  # Convert to Series
    X_val = pd.read_csv(X_val_path)
    y_val = pd.read_csv(y_val_path).squeeze()      # Convert to Series

except FileNotFoundError as e:
    print(e)
    raise

# Initialize the model with GPU support
model = XGBRegressor(
    n_estimators=100,          # Number of trees
    max_depth=10,              # Maximum depth of trees
    learning_rate=0.1,         # Learning rate
    subsample=0.8,             # Subsample ratio
    colsample_bytree=0.8,      # Feature sampling ratio
    tree_method="gpu_hist",    # Use GPU acceleration
    random_state=42            # Reproducibility
)

# Train the model
print("Training the model on GPU...")
model.fit(X_train, y_train)
print("Model training completed using GPU.")

# Optionally, evaluate the model
print("Evaluating the model...")
val_predictions = model.predict(X_val)
print(f"Validation Predictions: {val_predictions[:5]}")


Training the model on GPU...


c:\Users\theja\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:20:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Model training completed using GPU.
Evaluating the model...
Validation Predictions: [168.61284  116.47397   76.462875 104.39024  388.24426 ]


c:\Users\theja\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:20:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\theja\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:20:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the d

In [12]:
# Predict on validation set
y_val_pred = model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, y_val_pred)
rmse = mean_squared_error(y_val, y_val_pred, squared=False)
r2 = r2_score(y_val, y_val_pred)

print(f"Model Evaluation on Validation Set:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")


Model Evaluation on Validation Set:
Mean Absolute Error (MAE): 82.79
Root Mean Squared Error (RMSE): 153.34
R-squared (R2): 0.86


c:\Users\theja\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import os
import joblib

# Path to save the model
model_dir = "D:/mini project/Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning/models"
model_path = os.path.join(model_dir, "random_forest_model.pkl")

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Save the trained model
joblib.dump(model, model_path)
print(f"Model saved to {model_path}")

Model saved to D:/mini project/Prediction-and-Optimization-of-Smart-Dish-Specific-Demand-in-Restaurants-using-Machine-Learning/models\random_forest_model.pkl
